<a href="https://colab.research.google.com/github/AronPerez/CS4933_Project/blob/aaron/CV_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial 

## Aarons workspace

### Configs

In [32]:
!pip3 install --upgrade --force-reinstall tensorflow-gpu

     |████████████████████████████████| 394.3MB 31kB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 2.9MB 56.7MB/s 
     |████████████████████████████████| 133kB 58.6MB/s 
     |████████████████████████████████| 3.8MB 53.8MB/s 
     |████████████████████████████████| 14.8MB 56.7MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 6.0MB 56.6MB/s 
     |████████████████████████████████| 471kB 57.0MB/s 
     |████████████████████████████████| 1.0MB 55.2MB/s 
     |████████████████████████████████| 307kB 51.7MB/s 
     |████████████████████████████████| 788kB 55.5MB/s 
     |████████████████████████████████| 3.9MB 56.5MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 788kB 57.7MB/s 
     |████████████████████████████████| 153kB 59.4MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 

In [1]:
import os
import random
import numpy as np
import tensorflow as tf
import pickle
import glob
import matplotlib.pyplot as plt
from PIL import Image
from scipy.misc import imread
from abc import abstractmethod
from __future__ import print_function

In [2]:
# Prompt for input
name = input("Select a model name ")
while True:
  dataset = input("Select a dataset, either places365_dataset or cifar10_dataset ")
  if "PLACES365" in dataset.upper():
    dataset = 'PLACE365_DATASET'
    dataset_path = './dataset/places365'
    break
  elif 'CIFAR10' in dataset.upper():
    dataset = 'CIFAR10_DATASET'
    dataset_path = './dataset/cifar10'
    break
  else:
    print("Invalid dataset")
  
batch_size = int(input("Please select a batch size "))

options = {
    "seed": 100,
    "beta1": 0.0,
    "name": name.upper(),
    "mode": 0,
    "dataset": dataset,
    "dataset_path": dataset_path,
    "checkpoints_path": './checkpoints',
    "color_space": 'lab',
    "batch_size": batch_size,
    "l1-weigh": 100.0,
    "lr": '3e-4',
    "lr-decay": True,
    "lr-decay-rate": 0.1,
    "lr_decay_steps": '1e4',
    "augment": True,
    "acc_thresh": 2.0,
    "gpu_ids": 0,
    "save": True,
    "save_interval": 1000,
    "sample": True,
    "sample_size": 8,
    "sample_interval": 1000,
    "validate": False,
    "validate_interval": 0,
    "log": True,
    "log_interval": 10,
    "visualize": True,
    "visualize_window": 100,
    "test-input": '',
    "test-output": '',
    "turing_test_size": 100,
    "turing_test_delay": 0,
    "label_smoothing": 1
}

Select a model name Morty
Select a dataset, either places365_dataset or cifar10_dataset cifar10_dataset
Please select a batch size 60


### Driver

In [26]:
# Driver

# reset tensorflow graph
tf.compat.v1.reset_default_graph()
# initialize random seed
tf.random.set_seed(options["seed"])
np.random.seed(options["seed"])
random.seed(options["seed"])
# create a session environment
with tf.compat.v1.Session() as sess:
  if options["dataset"] == 'CIFAR10_DATASET':
    model = Cifar10Model(sess, options)

  elif options["dataset"] == 'PLACES365_DATASET':
    model = Places365Model(sess, options)

  if not os.path.exists(options["checkpoints_path"]):
    os.makedirs(options["checkpoints_path"])

  if options["log"]:
    open(model.train_log_file, 'w').close()
    open(model.test_log_file, 'w').close()
  # build the model and initialize
  model.build()
  sess.run(tf.global_variables_initializer())
  # load model only after global variables initialization
  model.load()
  if options.mode == 0:
    args = vars(options)
    print('\n------------ Options -------------')
    with open(os.path.join(options.checkpoints_path, 'options.dat'), 'w') as f:
      for k, v in sorted(args.items()):
        print('%s: %s' % (str(k), str(v)))
        f.write('%s: %s\n' % (str(k), str(v)))
    print('-------------- End ----------------\n') 

    model.train()
      
  elif options["mode"] == 1:
      model.test()
  else:
      model.turing_test()


NameError: ignored

### Dataset

In [5]:
CIFAR10_DATASET = 'cifar10'
PLACES365_DATASET = 'places365'

In [6]:
class BaseDataset():
    def __init__(self, name, path, training=True, augment=True):
        self.name = name
        self.augment = augment and training
        self.training = training
        self.path = path
        self._data = []

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        total = len(self)
        start = 0

        while start < total:
            item = self[start]
            start += 1
            yield item

        raise StopIteration

    def __getitem__(self, index):
        val = self.data[index]
        try:
            img = imread(val) if isinstance(val, str) else val

            # grayscale images
            if np.sum(img[:,:,0] - img[:,:,1]) == 0 and np.sum(img[:,:,0] - img[:,:,2]) == 0:
                return None

            if self.augment and np.random.binomial(1, 0.5) == 1:
                img = img[:, ::-1, :]

        except:
            img = None

        return img

    def generator(self, batch_size, recusrive=False):
        start = 0
        total = len(self)

        while True:
            while start < total:
                end = np.min([start + batch_size, total])
                items = []

                for ix in range(start, end):
                    item = self[ix]
                    if item is not None:
                        items.append(item)

                start = end
                yield items

            if recusrive:
                start = 0

            else:
                raise StopIteration

    @property
    def data(self):
        if len(self._data) == 0:
            self._data = self.load()
            np.random.shuffle(self._data)

        return self._data

    @abstractmethod
    def load(self):
        return []

In [7]:
class Cifar10Dataset(BaseDataset):
    def __init__(self, path, training=True, augment=True):
        super(Cifar10Dataset, self).__init__(CIFAR10_DATASET, path, training, augment)

    def load(self):
        data = []
        if self.training:
            for i in range(1, 6):
                filename = '{}/data_batch_{}'.format(self.path, i)
                batch_data = unpickle(filename)
                if len(data) > 0:
                    data = np.vstack((data, batch_data[b'data']))
                else:
                    data = batch_data[b'data']

        else:
            filename = '{}/test_batch'.format(self.path)
            batch_data = unpickle(filename)
            data = batch_data[b'data']

        w = 32
        h = 32
        s = w * h
        data = np.array(data)
        data = np.dstack((data[:, :s], data[:, s:2 * s], data[:, 2 * s:]))
        data = data.reshape((-1, w, h, 3))
        return data

### Helper

In [8]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise ('Boolean value expected.')

In [9]:
def parse(input):
  opt = input
  os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpu_ids
  opt.color_space = opt.color_space.upper()
  opt.training = opt.mode == 1

  if opt.seed == 0:
    opt.seed = random.randint(0, 2**31 - 1)

  if opt.dataset_path == './dataset':
    opt.dataset_path += ('/' + opt.dataset)

  if opt.checkpoints_path == './checkpoints':
    opt.checkpoints_path += ('/' + opt.dataset)

    return opt

### Utils

In [10]:
def stitch_images(grayscale, original, pred):
    gap = 5
    width, height = original[0][:, :, 0].shape
    img_per_row = 2 if width > 200 else 4
    img = Image.new('RGB', (width * img_per_row * 3 + gap * (img_per_row - 1), height * int(len(original) / img_per_row)))

    grayscale = np.array(grayscale).squeeze()
    original = np.array(original)
    pred = np.array(pred)

    for ix in range(len(original)):
        xoffset = int(ix % img_per_row) * width * 3 + int(ix % img_per_row) * gap
        yoffset = int(ix / img_per_row) * height
        im1 = Image.fromarray(grayscale[ix])
        im2 = Image.fromarray(original[ix])
        im3 = Image.fromarray((pred[ix] * 255).astype(np.uint8))
        img.paste(im1, (xoffset, yoffset))
        img.paste(im2, (xoffset + width, yoffset))
        img.paste(im3, (xoffset + width + width, yoffset))

    return img


In [11]:
def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

    return dir

In [12]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [13]:
def moving_average(data, window_width):
    cumsum_vec = np.cumsum(np.insert(data, 0, 0))
    ma_vec = (cumsum_vec[window_width:] - cumsum_vec[:-window_width]) / window_width
    return ma_vec

In [14]:
def imshow(img, title=''):
    fig = plt.gcf()
    fig.canvas.set_window_title(title)
    plt.axis('off')
    plt.imshow(img, interpolation='none')
    plt.show()

In [15]:
def imsave(img, path):
    im = Image.fromarray(np.array(img).astype(np.uint8).squeeze())
    im.save(path)

In [16]:
def turing_test(real_img, fake_img, delay=0):
    height, width, _ = real_img.shape
    imgs = np.array([real_img, (fake_img * 255).astype(np.uint8)])
    real_index = np.random.binomial(1, 0.5)
    fake_index = (real_index + 1) % 2

    img = Image.new('RGB', (2 + width * 2, height))
    img.paste(Image.fromarray(imgs[real_index]), (0, 0))
    img.paste(Image.fromarray(imgs[fake_index]), (2 + width, 0))

    img.success = 0

    def onclick(event):
        if event.xdata is not None:
            if event.x < width and real_index == 0:
                img.success = 1

            elif event.x > width and real_index == 1:
                img.success = 1

        plt.gcf().canvas.stop_event_loop()

    plt.ion()
    plt.gcf().canvas.mpl_connect('button_press_event', onclick)
    plt.title('click on the real image')
    plt.axis('off')
    plt.imshow(img, interpolation='none')
    plt.show()
    plt.draw()
    plt.gcf().canvas.start_event_loop(delay)

    return img.success

In [17]:
def visualize(train_log_file, test_log_file, window_width, title=''):
    train_data = np.loadtxt(train_log_file)
    test_data = np.loadtxt(test_log_file)

    if len(train_data.shape) < 2:
        return

    if len(train_data) < window_width:
        window_width = len(train_data) - 1

    fig = plt.gcf()
    fig.canvas.set_window_title(title)

    plt.ion()
    plt.subplot('121')
    plt.cla()
    if len(train_data) > 1:
        plt.plot(moving_average(train_data[:, 8], window_width))
    plt.title('train')

    plt.subplot('122')
    plt.cla()
    if len(test_data) > 1:
        plt.plot(test_data[:, 8])
    plt.title('test')

    plt.show()
    plt.draw()
    plt.pause(.01)

### Ops

In [18]:
COLORSPACE_RGB = 'RGB'
COLORSPACE_LAB = 'LAB'
#tf.nn.softmax_cross_entropy_with_logits_v2

In [19]:
def conv2d(inputs, filters, name, kernel_size=4, strides=2, bnorm=True, activation=None, seed=None):
    """
    Creates a conv2D block
    """
    initializer=tf.variance_scaling_initializer(seed=seed)
    res = tf.layers.conv2d(
        name=name,
        inputs=inputs,
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        padding="same",
        kernel_initializer=initializer)

    if bnorm:
        res = tf.layers.batch_normalization(inputs=res, name='bn_' + name, training=True)

    # activation after batch-norm
    if activation is not None:
        res = activation(res)

    return res

In [20]:
def conv2d_transpose(inputs, filters, name, kernel_size=4, strides=2, bnorm=True, activation=None, seed=None):
    """
    Creates a conv2D-transpose block
    """
    initializer=tf.variance_scaling_initializer(seed=seed)
    res = tf.layers.conv2d_transpose(
        name=name,
        inputs=inputs,
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        padding="same",
        kernel_initializer=initializer)

    if bnorm:
        res = tf.layers.batch_normalization(inputs=res, name='bn_' + name, training=True)

    # activation after batch-norm
    if activation is not None:
        res = activation(res)

    return res

In [21]:
def pixelwise_accuracy(img_real, img_fake, colorspace, thresh):
    """
    Measures the accuracy of the colorization process by comparing pixels
    """
    img_real = postprocess(img_real, colorspace, COLORSPACE_LAB)
    img_fake = postprocess(img_fake, colorspace, COLORSPACE_LAB)

    diffL = tf.abs(tf.round(img_real[..., 0]) - tf.round(img_fake[..., 0]))
    diffA = tf.abs(tf.round(img_real[..., 1]) - tf.round(img_fake[..., 1]))
    diffB = tf.abs(tf.round(img_real[..., 2]) - tf.round(img_fake[..., 2]))

    # within %thresh of the original
    predL = tf.cast(tf.less_equal(diffL, 1 * thresh), tf.float64)        # L: [0, 100]
    predA = tf.cast(tf.less_equal(diffA, 2.2 * thresh), tf.float64)      # A: [-110, 110]
    predB = tf.cast(tf.less_equal(diffB, 2.2 * thresh), tf.float64)      # B: [-110, 110]

    # all three channels are within the threshold
    pred = predL * predA * predB

    return tf.reduce_mean(pred)

In [22]:
def preprocess(img, colorspace_in, colorspace_out):
    if colorspace_out.upper() == COLORSPACE_RGB:
        if colorspace_in == COLORSPACE_LAB:
            img = lab_to_rgb(img)

        # [0, 1] => [-1, 1]
        img = (img / 255.0) * 2 - 1

    elif colorspace_out.upper() == COLORSPACE_LAB:
        if colorspace_in == COLORSPACE_RGB:
            img = rgb_to_lab(img / 255.0)

        L_chan, a_chan, b_chan = tf.unstack(img, axis=3)

        # L: [0, 100] => [-1, 1]
        # A, B: [-110, 110] => [-1, 1]
        img = tf.stack([L_chan / 50 - 1, a_chan / 110, b_chan / 110], axis=3)

    return img

In [23]:
def postprocess(img, colorspace_in, colorspace_out):
    if colorspace_in.upper() == COLORSPACE_RGB:
        # [-1, 1] => [0, 1]
        img = (img + 1) / 2

        if colorspace_out == COLORSPACE_LAB:
            img = rgb_to_lab(img)

    elif colorspace_in.upper() == COLORSPACE_LAB:
        L_chan, a_chan, b_chan = tf.unstack(img, axis=3)

        # L: [-1, 1] => [0, 100]
        # A, B: [-1, 1] => [-110, 110]
        img = tf.stack([(L_chan + 1) / 2 * 100, a_chan * 110, b_chan * 110], axis=3)

        if colorspace_out == COLORSPACE_RGB:
            img = lab_to_rgb(img)

    return img

In [24]:
def rgb_to_lab(srgb):
    # based on https://github.com/torch/image/blob/9f65c30167b2048ecbe8b7befdc6b2d6d12baee9/generic/image.c
    with tf.name_scope("rgb_to_lab"):
        srgb_pixels = tf.reshape(srgb, [-1, 3])

        with tf.name_scope("srgb_to_xyz"):
            linear_mask = tf.cast(srgb_pixels <= 0.04045, dtype=tf.float32)
            exponential_mask = tf.cast(srgb_pixels > 0.04045, dtype=tf.float32)
            rgb_pixels = (srgb_pixels / 12.92 * linear_mask) + (((srgb_pixels + 0.055) / 1.055) ** 2.4) * exponential_mask
            rgb_to_xyz = tf.constant([
                #    X        Y          Z
                [0.412453, 0.212671, 0.019334],  # R
                [0.357580, 0.715160, 0.119193],  # G
                [0.180423, 0.072169, 0.950227],  # B
            ])
            xyz_pixels = tf.matmul(rgb_pixels, rgb_to_xyz)

        # https://en.wikipedia.org/wiki/Lab_color_space#CIELAB-CIEXYZ_conversions
        with tf.name_scope("xyz_to_cielab"):

            # normalize for D65 white point
            xyz_normalized_pixels = tf.multiply(xyz_pixels, [1 / 0.950456, 1.0, 1 / 1.088754])

            epsilon = 6 / 29
            linear_mask = tf.cast(xyz_normalized_pixels <= (epsilon**3), dtype=tf.float32)
            exponential_mask = tf.cast(xyz_normalized_pixels > (epsilon**3), dtype=tf.float32)
            fxfyfz_pixels = (xyz_normalized_pixels / (3 * epsilon**2) + 4 / 29) * linear_mask + (xyz_normalized_pixels ** (1 / 3)) * exponential_mask

            # convert to lab
            fxfyfz_to_lab = tf.constant([
                #  l       a       b
                [0.0, 500.0, 0.0],  # fx
                [116.0, -500.0, 200.0],  # fy
                [0.0, 0.0, -200.0],  # fz
            ])
            lab_pixels = tf.matmul(fxfyfz_pixels, fxfyfz_to_lab) + tf.constant([-16.0, 0.0, 0.0])

        return tf.reshape(lab_pixels, tf.shape(srgb))

In [25]:
def lab_to_rgb(lab):
    with tf.name_scope("lab_to_rgb"):
        lab_pixels = tf.reshape(lab, [-1, 3])

        # https://en.wikipedia.org/wiki/Lab_color_space#CIELAB-CIEXYZ_conversions
        with tf.name_scope("cielab_to_xyz"):
            # convert to fxfyfz
            lab_to_fxfyfz = tf.constant([
                #   fx      fy        fz
                [1 / 116.0, 1 / 116.0, 1 / 116.0],  # l
                [1 / 500.0, 0.0, 0.0],  # a
                [0.0, 0.0, -1 / 200.0],  # b
            ])
            fxfyfz_pixels = tf.matmul(lab_pixels + tf.constant([16.0, 0.0, 0.0]), lab_to_fxfyfz)

            # convert to xyz
            epsilon = 6 / 29
            linear_mask = tf.cast(fxfyfz_pixels <= epsilon, dtype=tf.float32)
            exponential_mask = tf.cast(fxfyfz_pixels > epsilon, dtype=tf.float32)
            xyz_pixels = (3 * epsilon**2 * (fxfyfz_pixels - 4 / 29)) * linear_mask + (fxfyfz_pixels ** 3) * exponential_mask

            # denormalize for D65 white point
            xyz_pixels = tf.multiply(xyz_pixels, [0.950456, 1.0, 1.088754])

        with tf.name_scope("xyz_to_srgb"):
            xyz_to_rgb = tf.constant([
                #     r           g          b
                [3.2404542, -0.9692660, 0.0556434],  # x
                [-1.5371385, 1.8760108, -0.2040259],  # y
                [-0.4985314, 0.0415560, 1.0572252],  # z
            ])
            rgb_pixels = tf.matmul(xyz_pixels, xyz_to_rgb)
            # avoid a slightly negative number messing up the conversion
            rgb_pixels = tf.clip_by_value(rgb_pixels, 0.0, 1.0)
            linear_mask = tf.cast(rgb_pixels <= 0.0031308, dtype=tf.float32)
            exponential_mask = tf.cast(rgb_pixels > 0.0031308, dtype=tf.float32)
            srgb_pixels = (rgb_pixels * 12.92 * linear_mask) + ((rgb_pixels ** (1 / 2.4) * 1.055) - 0.055) * exponential_mask

        return tf.reshape(srgb_pixels, tf.shape(lab))